In [3]:
import pandas as pd
import json

In [4]:
file_name = "test_no-image.json"

with open(file_name, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df)

   page                                        information
0     0  [{'category_index': 0, 'category_name': 'title...
1     1  [{'category_index': 0, 'category_name': 'title...
2     2  [{'category_index': 0, 'category_name': 'title...
3     3  [{'category_index': 0, 'category_name': 'title...
4     4  [{'category_index': 0, 'category_name': 'title...
5     5  [{'category_index': 0, 'category_name': 'title...
6     6  [{'category_index': 2, 'category_name': 'plain...
7     7  [{'category_index': 0, 'category_name': 'title...
8     8  [{'category_index': 2, 'category_name': 'plain...
9     9  [{'category_index': 0, 'category_name': 'title...


In [5]:
data[0].keys()

dict_keys(['page', 'information'])

In [114]:
df = pd.DataFrame(data)
new_df = df
# you could try 
# col = df.map(type).astype(str).eq("<class 'list'>").all().idxmax()
# 'region_poly' would satisfy that condition but doesn't need to be expoloted. Besides, the structure should be stable
# Hence: column_names hard-code
cols = ['information','text_list']

for col in cols:
    print(col)
    new_df = new_df.explode(col).reset_index(drop=True)
    new_df = new_df.drop(columns=[col]).join(new_df[col].apply(pd.Series), rsuffix=f".{col}")

new_df

information
text_list


,page,category_index,category_name,region_poly,position,content,0
0,0,0,title,"[109, 1410, 230, 1410, 230, 1429, 109, 1429]","[108, 1410, 228, 1410, 228, 1426, 108, 1426]",[Introduction],NaN
1,0,0,title,"[108, 261, 510, 261, 510, 284, 108, 284]","[107, 258, 509, 259, 509, 283, 107, 282]",[ORIGINAL RESEARCH ARTICLE],NaN
2,0,0,title,"[100, 303, 989, 303, 989, 327, 100, 327]","[108, 299, 985, 302, 985, 327, 108, 325]",[Stingless bee distribution and richness in El...,NaN
3,0,0,title,"[100, 272, 988, 272, 988, 327, 100, 327]","[108, 299, 985, 302, 985, 327, 108, 325]",[Stingless bee distribution and richness in El...,NaN
4,0,2,plain text,"[152, 572, 1082, 572, 1082, 899, 152, 899]","[155, 566, 1079, 568, 1079, 586, 155, 584]",[This is the first complete inventory of sting...,NaN
...,...,...,...,...,...,...,...
1145,9,2,plain text,"[643, 266, 1133, 266, 1133, 352, 643, 352]","[673, 292, 1129, 290, 1129, 306, 673, 308]",[aguijon en Guatemala [Distribution of stingle...,NaN
1146,9,2,plain text,"[643, 266, 1133, 266, 1133, 352, 643, 352]","[673, 311, 1129, 310, 1129, 326, 673, 328]",[Guatemalal (5 pp). Guatemala city: Universida...,NaN
1147,9,2,plain text,"[643, 266, 1133, 266, 1133, 352, 643, 352]","[672, 331, 726, 331, 726, 347, 672, 347]",[Carlos.],NaN
1148,9,2,plain text,"[644, 227, 1133, 227, 1133, 269, 644, 269]","[642, 227, 1132, 226, 1132, 244, 642, 245]","[Wille, A. (1983). Biology of the stingless be...",NaN


In [12]:
file_outout_name = file_name.split(".")[0]+".csv"
print(file_outout_name)
new_df.to_csv(file_outout_name, header=True, index=False)

test_no-image.csv


### PARTE II: Dataframe to Document

In [120]:
# almost every error comes from not having the right data_type in column content. 
# It's very convenient to filter out problematic and non relevant categories from the beguinning
new_df.category_name.unique()
#new_df[new_df.category_name == 'figure']
#new_df[new_df.category_name == 'table']
#clean_df['content'] = new_df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
#clean_df['content']

array(['title', 'plain text', 'header', 'footnote', 'footer', 'figure',
       'page number', 'figure caption', 'table', 'table caption'],
      dtype=object)

In [181]:
df = pd.DataFrame(data)
new_df = df
# you could try 
# col = df.map(type).astype(str).eq("<class 'list'>").all().idxmax()
# 'region_poly' would satisfy that condition but doesn't need to be expoloted. Besides, the structure should be stable
# Hence: column_names hard-code
cols = ['information','text_list']

for col in cols:
    print(col)
    new_df = new_df.explode(col).reset_index(drop=True)
    new_df = new_df.drop(columns=[col]).join(new_df[col].apply(pd.Series), rsuffix=f".{col}")

new_df['content'] = new_df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
new_df['region_poly'] = new_df['region_poly'].apply(lambda x: tuple(x)) # lists are not mutable 
new_df['content_type'] = new_df['content'].apply(lambda x: type(x).__name__)

df_text = new_df[new_df.category_name == 'plain text']


information
text_list


In [183]:
aggr_level = ['page', 'region_poly']
grouped_df = df_text.groupby(aggr_level)

# Concatenate the content of the 'content' column with spaces
test = df_text.groupby(aggr_level)['content'].apply(lambda x: ' '.join(x)).reset_index()
test

,page,region_poly,content
0,0,"(102, 344, 1117, 344, 1117, 392, 102, 392)","Carlos Enrique Ruano lraheta**, Miguel Angel H..."
1,0,"(106, 523, 528, 523, 528, 543, 106, 543)",(Received 12 March 2011; accepted 18 December ...
2,0,"(106, 1437, 597, 1437, 597, 1566, 106, 1566)",El Salvador is the smallest country in Central...
3,0,"(108, 409, 1117, 409, 1117, 508, 108, 508)",0Focultod de Giengas Aeronomicas. Pebartamento...
4,0,"(152, 572, 1082, 572, 1082, 899, 152, 899)",This is the first complete inventory of stingl...
...,...,...,...
91,9,"(107, 291, 595, 291, 595, 372, 107, 372)","van Nieuwstadt, M. G. L., & Ruano Iraheta, C. ..."
92,9,"(108, 166, 593, 166, 593, 225, 108, 225)","Slaa, E. |. (2006). Population dynamics of a s..."
93,9,"(643, 266, 1133, 266, 1133, 352, 643, 352)","Yurrita, C., & Enriquez, E. (2005). Distribuci..."
94,9,"(644, 164, 1134, 164, 1134, 226, 644, 226)","Villanueya. R.. Roubik, D., & Colli-Ucan, W. (..."
